# Lexicon Creation

In [ ]:
#pip install symspellpy
from PyClasses.Processes.LexiconBuilder import LexiconBuilder
from pyspark.sql import SparkSession
from PyClasses.Neo4j.lexicon_nodes import LexiconNodeManager

uri = "neo4j+s://231511d9.databases.neo4j.io"
auth = ("neo4j", "m6fWF6qQUe6yGcZ81yAX0XG425OncyOB28A30vI9VvM") ## using zl personal account
spark = SparkSession.builder.appName("DE-prj").getOrCreate()
lexiconBuilder = LexiconBuilder(spark,uri,auth)
try:
    lnm = LexiconNodeManager(uri, auth)
    lexiconBuilder.build_lexicon()
    lexiconBuilder.build_peribahasa()
finally:
    pass

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
